## Day 82 Lecture 1 Assignment

In this assignment, we will learn about activation functions. We will create a neural network and measure the model's performance using different activations.

In [1]:
import numpy as np
import pandas as pd

We will import the famous titanic dataset below and produce a neural network that will predict the chance of survival for a passenger.

In [2]:
titanic = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/titanic.csv')

In [3]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


We'll perform some feature engineering

Let's start by keeping only the columns we'd like to use for our analysis. Keep only the columns: Survived, Pclass, Sex, SibSp, Parch, and Embarked

In [5]:
# Answer below:
df = titanic[['Survived', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Survived  891 non-null    int64 
 1   Pclass    891 non-null    int64 
 2   Sex       891 non-null    object
 3   SibSp     891 non-null    int64 
 4   Parch     891 non-null    int64 
 5   Embarked  889 non-null    object
dtypes: int64(4), object(2)
memory usage: 41.9+ KB


Now examine how many rows contain missing data. Given how much missing data we have, should we remove the column with the most missing data, or remove all rows containing missing data? Do what you think is best.

In [6]:
# Answer below: 
#remove rows containing missing data -- under Embarked
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Survived  889 non-null    int64 
 1   Pclass    889 non-null    int64 
 2   Sex       889 non-null    object
 3   SibSp     889 non-null    int64 
 4   Parch     889 non-null    int64 
 5   Embarked  889 non-null    object
dtypes: int64(4), object(2)
memory usage: 48.6+ KB


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Now we'll create a one hot encoding of the variables Pclass, sex, and Embarked

In [7]:
# Answer below:
#get_dummies -- Pclass, sex, Embarked
df_dummy = pd.get_dummies(df, columns=['Pclass', 'Sex', 'Embarked'], drop_first=True)
df_dummy.head()

,Survived,SibSp,Parch,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
0,0,1,0,0,1,1,0,1
1,1,1,0,0,0,0,0,0
2,1,0,0,0,1,0,0,1
3,1,1,0,0,0,0,0,1
4,0,0,0,0,1,1,0,1


Split the data into train and test. 20% of the data should be set aside for testing. Use Survived as your target variable.

In [8]:
# Answer below
from sklearn.model_selection import train_test_split

X = df_dummy.drop(['Survived'], axis=1)
y = df_dummy.Survived

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
X_train.shape

(711, 7)

At this point, we are ready to create a model. Import `Sequential` and `Dense` from Keras

In [10]:
# Answer below:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Create a model with 5 layers. The first layer should be a dense layer that receives the input, the last layer should be of size 1. You determine the remaining layer sizes.

Use a tanh activation for the output layer.

In [11]:
# Answer below
# 5 layers
#output layer size=1, activation='tanh'
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='tanh'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                512       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 4,193
Trainable params: 4,193
Non-trainable params: 0
_________________________________________________________________


Compile the model using the adam optimizer, binary crossentropy loss, and the accuracy metric.

Fit the model using a batch size of 80 over 200 epochs.

In [13]:
# Answer below:
#opt='adam', loss='binary_crossentropy'
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#fit model 
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, batch_size=80)

Epoch 1/200
9/9 [==============================] - 1s 64ms/step - loss: 1.2021 - accuracy: 0.5895 - val_loss: 0.8156 - val_accuracy: 0.6067
Epoch 2/200
9/9 [==============================] - 0s 6ms/step - loss: 0.7957 - accuracy: 0.6050 - val_loss: 0.7364 - val_accuracy: 0.6180
Epoch 3/200
9/9 [==============================] - 0s 6ms/step - loss: 0.7474 - accuracy: 0.5608 - val_loss: 0.7001 - val_accuracy: 0.6404
Epoch 4/200
9/9 [==============================] - 0s 6ms/step - loss: 0.7104 - accuracy: 0.5835 - val_loss: 0.6701 - val_accuracy: 0.6461
Epoch 5/200
9/9 [==============================] - 0s 6ms/step - loss: 0.6770 - accuracy: 0.6182 - val_loss: 0.6424 - val_accuracy: 0.6742
Epoch 6/200
9/9 [==============================] - 0s 6ms/step - loss: 0.6623 - accuracy: 0.6510 - val_loss: 0.6155 - val_accuracy: 0.6742
Epoch 7/200
9/9 [==============================] - 0s 6ms/step - loss: 0.6232 - accuracy: 0.6776 - val_loss: 0.5881 - val_accuracy: 0.6854
Epoch 8/200
9/9 [=========

In [14]:
history_df = pd.DataFrame(history.history)
history_df.head()

,loss,accuracy,val_loss,val_accuracy
0,1.004583,0.611814,0.815591,0.606742
1,0.775807,0.600563,0.736352,0.617977
2,0.732987,0.579466,0.700119,0.640449
3,0.701427,0.599156,0.670096,0.646067
4,0.670195,0.631505,0.642416,0.674157


In [15]:
model.evaluate(X_test, y_test)[1]

6/6 [==============================] - 0s 2ms/step - loss: 0.7764 - accuracy: 0.7809


0.7808988690376282

Redefine the model using a sigmoid activation for the last layer. What is the difference in accuracy.

In [16]:
# Answer below
# 5 layers
#output layer size=1, activation='sigmoid'
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 64)                512       
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_7 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_8 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 17        
Total params: 4,193
Trainable params: 4,193
Non-trainable params: 0
_________________________________________________________________


In [17]:
#opt='adam', loss='binary_crossentropy'
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#fit model 
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, batch_size=80)

Epoch 1/200
9/9 [==============================] - 1s 27ms/step - loss: 0.6375 - accuracy: 0.6166 - val_loss: 0.6069 - val_accuracy: 0.6124
Epoch 2/200
9/9 [==============================] - 0s 6ms/step - loss: 0.6106 - accuracy: 0.6043 - val_loss: 0.5804 - val_accuracy: 0.6124
Epoch 3/200
9/9 [==============================] - 0s 8ms/step - loss: 0.5797 - accuracy: 0.6216 - val_loss: 0.5601 - val_accuracy: 0.6124
Epoch 4/200
9/9 [==============================] - 0s 7ms/step - loss: 0.5688 - accuracy: 0.6071 - val_loss: 0.5460 - val_accuracy: 0.7191
Epoch 5/200
9/9 [==============================] - 0s 7ms/step - loss: 0.5246 - accuracy: 0.7029 - val_loss: 0.5336 - val_accuracy: 0.7191
Epoch 6/200
9/9 [==============================] - 0s 7ms/step - loss: 0.5292 - accuracy: 0.7547 - val_loss: 0.5265 - val_accuracy: 0.7753
Epoch 7/200
9/9 [==============================] - 0s 7ms/step - loss: 0.5377 - accuracy: 0.7886 - val_loss: 0.5172 - val_accuracy: 0.7865
Epoch 8/200
9/9 [=========

In [18]:
history_df = pd.DataFrame(history.history)
history_df.head()

,loss,accuracy,val_loss,val_accuracy
0,0.626170,0.618847,0.606867,0.612360
1,0.590533,0.618847,0.580412,0.612360
2,0.573193,0.618847,0.560147,0.612360
3,0.557143,0.630098,0.545991,0.719101
4,0.544402,0.704641,0.533623,0.719101


In [19]:
model.evaluate(X_test, y_test)[1]

6/6 [==============================] - 0s 2ms/step - loss: 0.5955 - accuracy: 0.7865


0.7865168452262878

In [ ]:
#accuracy is slightly better than activation='tanh' but not substantial